In [ ]:
#!/usr/bin/env bash

<span style="color: red; font-weight: bold">Use the following command to launch a server for this notebook:</span>

```bash
git clone git@github.com:input-output-hk/marlowe-cardano.git
cd marlowe-cardano/marlowe-runtime/
nix run ../marlowe-cli
```

Then navigate to the `examples/` folder in Jupyter and open this notebook.

# Demonstrating the Marlowe Transaction Creation Component of Marlowe Runtime

## Preliminaries

Record version numbers

In [ ]:
marlowe-cli --version

In [ ]:
cardano-cli --version

In [ ]:
git rev-parse HEAD

### Setup the faucet.

Set the location of keys.

In [1]:
TREASURY=treasury

Set the faucet.

In [2]:
FAUCET_SKEY=$TREASURY/payment.skey
FAUCET_ADDR=$(cat $TREASURY/payment.testnet.address)
echo "$FAUCET_ADDR"

addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


### Select network

In a separate terminal, set up a tunnel to the Marlowe Runtime development server:
```bash
rm /tmp/preview.socket
ssh -NT \
  -L/tmp/preview.socket:/data/networks/preview/node.socket \
  -L 3717:127.0.0.1:23717 \
  -L 3718:127.0.0.1:23718 \
  -L 3719:127.0.0.1:23719 \
  -L 3721:127.0.0.1:23721 \
  -L 3723:127.0.0.1:23723 \
  54.202.238.5                                              
```

In [3]:
export CARDANO_NODE_SOCKET_PATH=/tmp/preview.socket
export CARDANO_TESTNET_MAGIC=2
MAGIC=(--testnet-magic 2)
echo "${MAGIC[@]}"

--testnet-magic 2


### Time computations

In [4]:
SECOND=1000
MINUTE=$((60 * SECOND))
HOUR=$((60 * MINUTE))

In [5]:
NOW="$(($(date -u +%s) * SECOND))"
echo "$NOW"

1665800628000


### Check that the reference script has been published

Check that the Marlowe semantics validator was published.

In [6]:
marlowe-cli transaction find-published


Searching for reference script at address: addr_test1vrw0tuh8l95thdqr65dmpcfqnmcw0en7v7vhgegck7gzqgswa07sw

Expected reference script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Searching for reference script at address: addr_test1vpa36uuyf95kxpcleldsncedlhjru6vdmh2vnpkdrsz4u6cll9zas

Expected reference script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"
{
    "marlowe": {
        "hash": "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#1"
    },
    "payout": {
        "hash": "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#2"
    }
}


### Participants

#### The Party

In [7]:
PARTY_SKEY="$TREASURY/john-fletcher.skey"
PARTY_VKEY="$TREASURY/john-fletcher.vkey"

Create the first party's keys, if necessary.

In [8]:
if [[ ! -e "$PARTY_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$PARTY_SKEY" --verification-key-file "$PARTY_VKEY"
fi
PARTY_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$PARTY_VKEY")
echo "$PARTY_ADDR"

addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r


Fund the address.

In [9]:
marlowe-cli util fund-address \
  --out-file /dev/null \
  --submit 600 \
  --lovelace 250000000 \
  --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
  "$PARTY_ADDR"

TxId "b366e1653f3f654df9d3272a3b634f549deae5c9a7abb7377e891745b834d661"


#### The Counterparty

In [10]:
COUNTERPARTY_SKEY="$TREASURY/thomas-kyd.skey"
COUNTERPARTY_VKEY="$TREASURY/thomas-kyd.vkey"

Create the second party's keys, if necessary.

In [11]:
if [[ ! -e "$COUNTERPARTY_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$COUNTERPARTY_SKEY" --verification-key-file "$COUNTERPARTY_VKEY"
fi
COUNTERPARTY_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$COUNTERPARTY_VKEY")
echo "$COUNTERPARTY_ADDR"

addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5


Fund the address.

In [12]:
marlowe-cli util fund-address \
  --out-file /dev/null \
  --submit 600  \
  --lovelace 250000000 \
  --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
  "$COUNTERPARTY_ADDR"

TxId "9489a85c974e6206b7e4d6a6cb473e6a5c266c0ad5583047e8a7a48f04138441"


## The Contract

We set the parameters for the ACTUS PAM contract.

In [13]:
MINIMUM_ADA=3000000

FIXED_POINT=1000000
PRINCIPAL=100
INTEREST_RATE=0.02

STATUS_DATE=$(date -d "$(date -u -R -d @$((NOW/1000)))" +"%Y-%m-%dT00:00:00")
INITIAL_EXCHANGE_DATE=$(date -d "$(date -u -R -d @$((NOW/1000))) + 1 year" +"%Y-01-01T00:00:00")
MATURITY_DATE=$(date -d "$(date -u -R -d @$((NOW/1000))) + 2 year" +"%Y-01-01T00:00:00")

#LENDING_DEADLINE=$((NOW+12*HOUR))
#REPAYMENT_DEADLINE=$((NOW+24*HOUR))

In [14]:
yaml2json << EOI > history.actus
scheduleConfig:
  businessDayConvention: "NULL"
  endOfMonthConvention: "EOM"
  calendar: "NC"
maturityDate: "$MATURITY_DATE"
contractId: "0"
enableSettlement: false
initialExchangeDate: "$INITIAL_EXCHANGE_DATE"
contractRole: "RPA"
penaltyType: "O"
cycleAnchorDateOfInterestPayment: "$INITIAL_EXCHANGE_DATE"
contractType: "PAM"
notionalPrincipal: $PRINCIPAL
contractPerformance: "PF"
collateralAmount: 0
dayCountConvention: "30E360"
accruedInterest: 0
statusDate: "$STATUS_DATE"
cycleOfInterestPayment: "P1YL1"
prepaymentEffect: "N"
nominalInterestRate: $INTEREST_RATE
interestCalculationBase: "NT"
EOI
cat history.actus

{"accruedInterest":0,"collateralAmount":0,"contractId":"0","contractPerformance":"PF","contractRole":"RPA","contractType":"PAM","cycleAnchorDateOfInterestPayment":"2023-01-01T00:00:00","cycleOfInterestPayment":"P1YL1","dayCountConvention":"30E360","enableSettlement":false,"initialExchangeDate":"2023-01-01T00:00:00","interestCalculationBase":"NT","maturityDate":"2024-01-01T00:00:00","nominalInterestRate":0.02,"notionalPrincipal":100,"penaltyType":"O","prepaymentEffect":"N","scheduleConfig":{"businessDayConvention":"NULL","calendar":"NC","endOfMonthConvention":"EOM"},"statusDate":"2022-10-14T00:00:00"}


Create the contract.

In [15]:
marlowe-cli template actus \
  --minimum-ada "$MINIMUM_ADA" \
  --party "$PARTY_ADDR" \
  --counter-party "$COUNTERPARTY_ADDR" \
  --actus-terms-file  history.actus \
  --out-contract-file create-1.contract \
  --out-state-file /dev/null

View the contract.

In [16]:
json2yaml create-1.contract

timeout: 1672531200000
timeout_continuation: close
when:
- case:
    deposits:
      negate:
        negate: 100000000
    into_account:
      address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
  then:
    from_account:
      address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
    pay: 100000000
    then:
      timeout: 1704067200000
      timeout_continuation: close
      when:
      - case:
          deposits: 2000000
          into_account:
            address: addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5
          of_token:
            currency_symbol: ''
            token_name: ''
          party:
            address: addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5
        then:
          from_account:
            address: addr_test1vr7n0zzth5zycuh972w7rdm

## Run the Contract

### Transaction 1. Create the ontract

See what UTxOs the transaction-creation will have available to select from.

In [17]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
b366e1653f3f654df9d3272a3b634f549deae5c9a7abb7377e891745b834d661     1        250000000 lovelace + TxOutDatumNone


Build the transaction.

In [18]:
CONTRACT_ID=$(
marlowe create \
  --core-file create-1.contract \
  --min-utxo "$MINIMUM_ADA" \
  --change-address "$PARTY_ADDR" \
  --address "$PARTY_ADDR" \
  --manual-sign create-1.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "CONTRACT_ID = $CONTRACT_ID"

CONTRACT_ID = 2cc4bcb35f8b86836eef45d77ce22d467170d5749fbb8ce48f49fd43d6bcef20#1


Sign the transaction.

In [19]:
cardano-cli transaction sign \
  --tx-body-file create-1.txbody \
  --out-file     create-1.tx \
  --signing-key-file "$PARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [20]:
marlowe submit create-1.tx

BlockHeader {slotNo = SlotNo {unSlotNo = 5797471}, headerHash = "2a155b298c80936ff42d0dca715941f6058e2d78b53565ba74373652eacbade5", blockNo = BlockNo {unBlockNo = 267167}}


Watch the contract.

In [21]:
marlowe add "$CONTRACT_ID"

2cc4bcb35f8b86836eef45d77ce22d467170d5749fbb8ce48f49fd43d6bcef20#1


In [22]:
marlowe log --show-contract "$CONTRACT_ID"

transaction 2cc4bcb35f8b86836eef45d77ce22d467170d5749fbb8ce48f49fd43d6bcef20 (creation)
ContractId:      2cc4bcb35f8b86836eef45d77ce22d467170d5749fbb8ce48f49fd43d6bcef20#1
SlotNo:          5797471
BlockNo:         267167
BlockId:         2a155b298c80936ff42d0dca715941f6058e2d78b53565ba74373652eacbade5
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Deposit Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r" Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"
            (Token "" "")
            (NegValue
               (NegValue
                  (Constant 100000000))))
         (Pay Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"
            (Party Address "addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5")
            (Token "" "")
            (Constant 100000000)
            (When [
               (Case
          

View the contract's UTxO.

In [23]:
CONTRACT_ADDRESS=$(marlowe-cli contract address)
echo "$CONTRACT_ADDRESS"

addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv


In [24]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDRESS" | sed -n -e "1,2p;/${CONTRACT_ID//#*/}/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
2cc4bcb35f8b86836eef45d77ce22d467170d5749fbb8ce48f49fd43d6bcef20     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "6974dab12c1ebf84b28387ef0768e6e14f2052533fa5fad52386a33ad6532735"


### Transaction 2. Party deposits loan amount

In [25]:
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$PARTY_ADDR" \
  --to-party "$PARTY_ADDR" \
  --lovelace "$((PRINCIPAL*FIXED_POINT))" \
  --validity-lower-bound "$((NOW - 5 * MINUTE))" \
  --validity-upper-bound "$((NOW + 1 * HOUR))" \
  --change-address "$PARTY_ADDR" \
  --address "$PARTY_ADDR" \
  --manual-sign create-2.txbody

marlowe: not implemented
CallStack (from HasCallStack):
  error, called at cli/Language/Marlowe/Runtime/CLI/Command/Apply.hs:191:19 in main:Language.Marlowe.Runtime.CLI.Command.Apply


: 1

<span style="color:red;font-weight:bold">It looks like there is a deserialization problem when applying inputs.</span>

### Transaction 3. Counterparty repays the loan's interest

In [ ]:
#marlowe-cli run prepare \
#  --marlowe-file create-2.marlowe \
#  --out-file     create-3.marlowe \
#  --deposit-account "$COUNTERPARTY_ADDR" \
#  --deposit-party "$COUNTERPARTY_ADDR" \
#  --deposit-amount "$((INTEREST*FIXED_POINT))" \
#  --invalid-before "$((NOW-5*MINUTE))" \
#  --invalid-hereafter "$((NOW+1*HOUR))" \
#  --print-stats

### Transaction 4. Counterparty repays the loan's principal

In [ ]:
#marlowe-cli run prepare \
#  --marlowe-file create-3.marlowe \
#  --out-file     create-4.marlowe \
#  --deposit-account "$COUNTERPARTY_ADDR" \
#  --deposit-party "$COUNTERPARTY_ADDR" \
#  --deposit-amount "$((PRINCIPAL*FIXED_POINT))" \
#  --invalid-before "$((NOW-5*MINUTE))" \
#  --invalid-hereafter "$((NOW+4*HOUR))" \
#  --print-stats

The contract has closed.

## Cleanup

Remove the contract from history tracking.

In [26]:
marlowe rm "$CONTRACT_ID"

2cc4bcb35f8b86836eef45d77ce22d467170d5749fbb8ce48f49fd43d6bcef20#1


In [27]:
marlowe ls

Consolidate the UTxOs at the addresses.

In [28]:
marlowe-cli util clean \
  --change-address "$PARTY_ADDR" \
  --required-signer "$PARTY_SKEY" \
  --out-file /dev/null \
  --submit 600

TxId "b066f7536f4dea7bc04750c92879ad65362270e1035772b59c03ad89aa1d666c"


In [29]:
marlowe-cli util clean \
  --change-address "$COUNTERPARTY_ADDR" \
  --required-signer "$COUNTERPARTY_SKEY" \
  --out-file /dev/null \
  --submit 600

TxId "419da57b22ac4a74246d88e6ba16a0ae1b9315aaa42b4406e276290b3069ccaa"


Send the funds back to the faucet.

In [30]:
marlowe-cli transaction simple \
  --tx-in "$(marlowe-cli util select --lovelace-only 1 "$PARTY_ADDR" | sed -n -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;1p')" \
  --tx-in "$(marlowe-cli util select --lovelace-only 1 "$COUNTERPARTY_ADDR" | sed -n -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;1p')" \
  --required-signer "$PARTY_SKEY" \
  --required-signer "$COUNTERPARTY_SKEY" \
  --change-address "$FAUCET_ADDR" \
  --out-file /dev/null \
  --submit 600

TxId "21c5c05f0749277e2c1c66ecd6623927567ffb1f7ecdcc5f5120278b08180759"


In [31]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR" --address "$COUNTERPARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
